# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [7]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import processingmm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# set the parameters to run the script
directories = [r'/media/elea/ssd/NPP/test']
calib_directory = r'/media/elea/ssd/NPP/calib/'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'pddn'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550,600]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz
# NB: processing time without PDDN (with the data on a SSD drive)
# 'no_viz': 0.39s
# 'default', save_pdf_figs False: 1.65s
# 'default', save_pdf_figs True: 2.67s
# 'full', save_pdf_figs False: 3.24s
# 'full', save_pdf_figs True: 5.94s
processing_mode = 'default'
save_pdf_figs = False

# define which intrument is being used (currently supported 'IMP', 'IMP_v2')
instrument = 'IMP'
denoise_patch = True

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = True

# define if the specular reflections should be removed
remove_reflection = False

parameters = processingmm.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, instrument = instrument, remove_reflection = remove_reflection,
                                processing_mode = processing_mode, run_all = run_all, save_pdf_figs = save_pdf_figs, 
                                align_wls = align_wls, denoise_patch = denoise_patch)

 [info] Calibration directory /media/elea/ssd/NPP/calib/ will not be used.
 [info] Processing data from the IMP instrument.
 [info] Switching denoise_patch to False.


In [10]:
%%time
times = processingmm.batch_process_master(parameters)
times

Processing with PDDN...

Loading PDDN models...
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- ...
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- Complete
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- ...
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- Complete
Loading PDDN models done.


Denoising inference...


100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 18558.87it/s]


Denoising inference done.

Aligning wavelength: 600 nm...


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3663.15it/s]


Aligning wavelengths done.



  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: /media/elea/ssd/NPP/test/2022-07-06_T_AUTOPSY-BF_FR_S1_1


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.86s/it]

Processing with PDDN done.
CPU times: user 4.53 s, sys: 376 ms, total: 4.9 s
Wall time: 2.53 s


{'azimuth_curation': 0.005285978317260742,
 'data_loading': 0.14579272270202637,
 'MM_processing': 0.3520636558532715,
 'azimuth_std_processing': 0.046418190002441406,
 'full_processing': 0.5497760772705078,
 'save_npz': 0.1636807918548584,
 'viz': 1.1407709121704102,
 'total': 1.8587348461151123}

In [12]:
from processingmm.addons import predict
predict.batch_prediction(parameters, no_labels = False, MM = False, model_name = None)

INFO: Using device cuda
INFO: Model loaded from intensities_1.pt
Steps 1 test:   0%|          | 0/1 [00:00<?, ?img/s]

Processing with labels...

Running predictions on sample 2022-07-06_T_AUTOPSY-BF_FR_S1_1
 [wrn] No metadata found in /media/elea/ssd/NPP/test/clinical_data.xlsx


Steps 1 test: 100%|██████████| 1/1 [00:01<00:00,  1.65s/img]


In [22]:
%%time
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
run_all = False
times = visualization_lines.batch_visualization(parameters, run_all)
times

 [wrn] Visualization was not optimized to support IMPv2.
CPU times: user 1.05 ms, sys: 6 μs, total: 1.06 ms
Wall time: 843 μs


0.0007061958312988281

In [26]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
run_all = False
align_wavelengths.align_wavelengths(parameters['directories'], parameters['PDDN'], run_all, [600], parameters['instrument'])

NotImplementedError: The alignment of wavelengths is not implemented for IMPv2.